In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import os 
import seaborn as sns
import glob
import natsort

In [ ]:
import seaborn as sns
import copy

def load_data(save_path):
    datas = list()
    path_list = glob.glob(save_path)
    path_list = natsort.natsorted(path_list)
    data_temp = get_concat_glob(path_list)
    data_temp["a1"] = data_temp["a1"] * 100
    data_temp["a3"] = data_temp["a3"] * 100
    data_temp['Distribution'] = data_temp['distribution'] .map({0: "IID Label Distribution", 
                                                                0.3: "Non-IID Label Distribution",
                                                                0.5: "Non-IID Label Distribution"})
    return data_temp

In [ ]:
def get_concat_glob(paths):
    data_ = []
    for i, path in enumerate(paths):


        # =====================================
        if "GGU" in path: continue
        if "GU211" in path: continue

        # if "EP10" not in path: continue
        # =====================================

       
        data = pd.read_csv(path)
        data["Mode"] = data["mode"]
        data["Scaling"] = "Vanilla"
        data["GU_ratio"] = 0
        data["n_epochs"] = 5

        if "_GU111.02" in path and "CIFAR100-" in path: 
            data["Scaling"] = "GU111"
        elif "_GU112.02" in path and "CIFAR100-" in path: 
            data["Scaling"] = "GU112"
        elif "_GU111.04" in path and "CIFAR10-" in path: 
            data["Scaling"] = "GU111"
        elif "_GU112.04" in path and "CIFAR10-" in path: 
            data["Scaling"] = "GU112"
        elif "_GU111.04" in path and "TIN" in path: 
            data["Scaling"] = "GU111"
        elif "_GU112.04" in path and "TIN" in path: 
            data["Scaling"] = "GU112"
        elif "_GU311" in path: 
            data["Scaling"] = "GU311"
        elif "_GU312" in path: 
            data["Scaling"] = "GU312"
        elif "_GU1" in path:
            continue
        elif "_GU2" in path:
            continue

        if "EP10" in path: data["n_epochs"] = 10
        
        if len(data_) == 0: data_ = data
        else: data_ = pd.concat([data_, data])
    return data_

## BarPlot

In [ ]:
data_c100 = load_data("../Output_GU_C100/CIFAR100-*-N100-*/s[1-9].csv")   
data_c10  = load_data("../Output_GU_C10_M9/CIFAR10-*-N100-*/s[1-9].csv")
data_tin  = load_data("../Output_GU_TIN/TinyImageNet-*-N100-*/s[1-9].csv")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3), dpi=150, sharey=True)
sns.set_style("darkgrid")
for index, task, data_temp, title  in zip(range(3), ["Cifar10", "Cifar100", "TIN"], [data_c10, data_c100, data_tin], ["CIFAR-10", "CIFAR-100",  "Tiny-ImageNet"]):
    ax = axes[index]
    temp = copy.deepcopy(data_temp)
    temp = temp[temp["act_prob"] == 0.01]
    if index == 0: temp = temp[temp["n_epochs"] == 10]
    elif index in [1,2]: temp = temp[temp["n_epochs"] == 5]
    if   index in [0]: temp = temp[(temp["epoch"] == 599)]
    elif index in [1]: temp = temp[(temp["epoch"] == 599)]
    elif index in [2]: temp = temp[(temp["epoch"] == 399)]

    replace_icon = {
        "GU111": "FedBug: Bottom-Up GU",
        "GU112": "Top-Down GU",
        "GU311": "FedBABU: Fix the Last Layer",
        "GU312": "Fix Last Two Layers",
    }
    sk = ["Vanilla"] + list(replace_icon.keys())
    temp = temp.set_index('Scaling').loc[sk].reset_index().replace({"Scaling": replace_icon})
    
    temp = temp[(temp["mode"] == "fedavg")]
    ax = sns.barplot(data=temp, x="a3", y="Scaling", hue="Distribution", ax=ax, errorbar="se", linewidth = 0.8)
    ax.set_title(title)

axes[0].set_xlabel("Test Accuracy (%)"), axes[1].set_xlabel("Test Accuracy (%)")
axes[0].set_xlim([75,83]), axes[1].set_xlim([35,51]), axes[2].set_xlim([5,28])
axes[0].get_legend().remove(), axes[1].get_legend().remove(), axes[2].legend(loc=4, prop={'size': 8})
axes[0].tick_params(axis='y', which='minor', pad=5)

for ax in axes.flatten(): ax.set_ylabel("")
fig.tight_layout()
plt.savefig("../SaveFig/exp_ablation_fixation.pdf")
plt.pause(0.1)

NameError: name 'plt' is not defined

In [ ]:
data_c100

,Unnamed: 0,task,mode,balance,distribution,n_clients,act_prob,seed,epoch,a1,...,a4,l1,l2,l3,l4,Mode,Scaling,GU_ratio,n_epochs,Distribution
0,0,CIFAR100,fedadagrad,0,0.0,100,0.01,1,9,8.85,...,0.03614,4.084081,4.058978,4.584213,4.583886,fedadagrad,Vanilla,0,5,IID Label Distribution
1,1,CIFAR100,fedadagrad,0,0.0,100,0.01,1,19,13.72,...,0.07264,4.021590,3.945961,4.539647,4.538213,fedadagrad,Vanilla,0,5,IID Label Distribution
2,2,CIFAR100,fedadagrad,0,0.0,100,0.01,1,29,15.04,...,0.09762,4.013478,3.896148,4.495186,4.492257,fedadagrad,Vanilla,0,5,IID Label Distribution
3,3,CIFAR100,fedadagrad,0,0.0,100,0.01,1,39,17.54,...,0.13264,3.955952,3.789112,4.401631,4.395136,fedadagrad,Vanilla,0,5,IID Label Distribution
4,4,CIFAR100,fedadagrad,0,0.0,100,0.01,1,49,18.70,...,0.18108,3.876583,3.662413,4.209012,4.193949,fedadagrad,Vanilla,0,5,IID Label Distribution
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,47,CIFAR100,scaffold,0,0.3,100,0.01,2,479,1.44,...,0.01444,4.596143,4.595134,4.596482,4.595521,scaffold,GU111,0,5,Non-IID Label Distribution
48,48,CIFAR100,scaffold,0,0.3,100,0.01,2,489,1.43,...,0.01452,4.596119,4.595129,4.596465,4.595502,scaffold,GU111,0,5,Non-IID Label Distribution
49,49,CIFAR100,scaffold,0,0.3,100,0.01,2,499,1.41,...,0.01460,4.596138,4.595154,4.596416,4.595454,scaffold,GU111,0,5,Non-IID Label Distribution
50,50,CIFAR100,scaffold,0,0.3,100,0.01,2,509,1.42,...,0.01456,4.596082,4.595104,4.596356,4.595392,scaffold,GU111,0,5,Non-IID Label Distribution
